# The Notebook: Segmenting and Clustering Neighborhoods in Toronto

#### Assignment written by Meng Chen

## The general instruction of this assignment

1. Scape the Wikipedia webpage: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:M to obtain the neighborhoods in Toronto in a table.
2. Transform the data into a pandas dataframe. 
3. The dataframe should be formated as:
   a) Three columns: PostalCode, Borough, and Neighborhood;
   b) Remove cells with a borough that is not assigned;
   c) Merge the neighborhoods sharing same PostalCode
   d) If a cell has a borough but a NOT assigned neighborhood, then the neighborhood will be same as the borough
4. Get the latitude and longtitude data of those neighborhood:         
   a) Google Maps Geocoding API was a good option until it started to charge for using their API;        
   b) Alternatively, in this assignment, we will use Geocode Python package instead (https://geocoder.readthedocs.io/index.html): i) the issue of above Python 
      package is not consistently producing the coordinates; ii) To solve this issue, I will run the function provided by the instructor;        
   c) After data acquisation, create a dataframe.
5. Finally, explore and cluster the neighborhoods in Toronto based on preivous lab execrises on New York City.
6. The assignment should be divided into three notebooks for submission.

## 1. Scape the Wikipedia webpage and transform the data into pandas dataframe

In [6]:
# install neccessory libraries
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

In [248]:
# import neccessory libraries
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd

print("Library imported --- Done !")

Library imported --- Done !


In [249]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# create a handle, page, to handle the content of the website
page =requests.get(url)

# store the content of the website under web
web = lh.fromstring(page.content)

# parse data that are store between <tr>...</tr> of HTML
tr_elements = web.xpath('//tr')

# chekc the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [250]:
# Parse table header
tr_elements = web.xpath('//tr')

# creat empty list
col=[]
i=0

# for each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print (i, name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [251]:
# create pandas dataframe

# since our first row is the header, data is tore on the second row onwards
for j in range (1, len(tr_elements)):
    # T is our j'th row
    T=tr_elements[j]
    
    # if row is not of size 3, the //tr data is not from our table
    if len(T)!=3:
        break
    
    # i is the index of our column
    i = 0
    
    # iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        # append the data to empty list of the i'th column
        col[i][1].append(data)
        # increment i for the next column
        i+=1
        
print ("The process is finished! ")

The process is finished! 


In [252]:
Dict={title:column for (title, column) in col}

In [253]:
web=pd.DataFrame(Dict)
web.shape

(289, 3)

In [254]:
web.columns =['PostCode', 'Borough', 'Neighborhood'] # rename the column after making dataframe
web.replace({'Neighborhood': {"\n":""}}, regex =True, inplace=True) # remove unwanted "/n" in Neighborhood column
web.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove unwanted cells in the dataframe

In [255]:
# remove the Not assigned in Borough
web = web[web.Borough != 'Not assigned']

# reset the index because dropoff rows
web.reset_index(drop = True, inplace = True)

In [256]:
web.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [257]:
# merge the neighborhoods sharing same postcodes
web_group = web.groupby(['PostCode','Borough'], as_index = False).agg({'Neighborhood': lambda x: ', '.join(x)})
web_group.shape

(103, 3)

In [277]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
g=0

for g in range(web_group.shape[0]):
    if web_group['Neighborhood'][g] == 'Not assigned':
        web_group['Neighborhood'][g] = web_group['Borough'][g]
    elif web_group['Neighborhood'][g] != 'Not assigned':
            continue
            
web_group.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## 2. Attain the GPS coordiantes of those Neighborhoods


In [279]:
# install the geocoder library
!pip install geocoder
print ('Done')

Done


In [295]:
# import the geocoder library
import geocoder

# import other libraries
import pandas as pd
import numpy as np

print("Library imported --- Done !")

Library imported --- Done !


In [ ]:
# apply the function provided by instructors

postal_code =  web_group['PostCode']

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print ("Done")

### The geocoder is unstable and just kept my python kernal running without producing results. Thus, I used the provided coordinate file

In [297]:
filename = 'http://cocl.us/Geospatial_data'

coords = pd.read_csv(filename)

In [325]:
# Merge two dataframes together
web_group_all = web_group.merge(coords, right_on = "Postal Code", left_on = 'PostCode', how = 'inner')
# Drop the extra postal code column
web_group_all.drop("Postal Code", axis = 1, inplace = True)
# check if any rows were removed during the merge
web_group.shape
# show the first five rows
web_group_all.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Explore and cluster the neighborhoods in Toronto

In [326]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.17.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-certificates: 2018.03.07-0                  --> 2018.11.29-ha4d7672_0 conda-forge
    certifi:         2018.11.29-py36_0             --> 2018.11.29-py36_1000  conda-forge
    conda:           4.5.11-py36_0                 --> 4.5.11-py36_1000      

In [327]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(web_group_all['Borough'].unique()),
        web_group_all.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [329]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [330]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(web_group_all['Latitude'], web_group_all['Longitude'], web_group_all['Borough'], web_group_all['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [331]:
scarborough_data = web_group_all[web_group_all['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [338]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Define Foursquare Credentials and Version

In [339]:
CLIENT_ID = 'QBJKCK54JEFMRBEM2PYPPCZJAOPWKDTXJPLBUAOPJTW00USE' # your Foursquare ID
CLIENT_SECRET = 'BKF5XYC4KEG2R1WCRBFAEJJOVXLKO3SE0PVYQL2HVPFFHN54' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
radius = 500

### Use the function below to explore scarborough

In [341]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [342]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [343]:
# Check the size of resulting dataframe
print (scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [344]:
scarborough_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",9,9,9,9,9,9
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


In [346]:
print ('There are {} unique categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 56 unique categories.


### Analyze each neighborhood

In [347]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [348]:
# new dataframe shape
scarborough_onehot.shape

(89, 57)

In [390]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.200000,0.0,0.000000,0.00,0.000000,0.200000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.2,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.200000,0.000,0.20,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000,0.5,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.25,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.142857,0.000,0.142857,0.142857,0.0,0.000000,0.0,0.000000,0.00,0.142857,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.142857,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.000,0.0,0.000000,0.142857,0.000000,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.000,0.200000,0.000000,0.0,0.000000,0.2,0.100000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.1,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.1,0.000000,0.000,0.0,0.000000,0.000,0.1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.111111,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.111111,0.111111,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.111111,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.0,0.111111,0.000,0.0,0.000000,0.111111,0.222222,0.0,0.0,0.000000,0.000000,0.000,0.00,0.000,0.0,0.000000,0.111111,0.000000,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.5,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000

In [391]:
# Print the each neighborhood along the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3  Chinese Restaurant   0.2
4      Sandwich Place   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2        American Restaurant   0.0
3                   Pharmacy   0.0
4  Latin American Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0  Fried Chicken Joint  0.14
1               Bakery  0.14
2                 Bank  0.14
3   Athletics & Sports  0.14
4      Thai Restaurant  0.14


----Clairlea, Golden Mile, Oakridge----
                  venue  freq
0                Bakery   0.2
1              Bu

In [392]:
# create function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [393]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,History Museum,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Athletics & Sports,Thai Restaurant,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Cosmetics Shop,History Museum
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Intersection,Metro Station,Fast Food Restaurant,Bus Station,Park,Soccer Field,General Entertainment,Cosmetics Shop
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Italian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Noodle House,Pharmacy,Chinese Restaurant,Thai Restaurant,Bar,Convenience Store
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Intersection,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Bank,Bar,History Museum,Hakka Restaurant,Athletics & Sports
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Bus Station,Department Store,Coffee Shop,Train Station,Vietnamese Restaurant,Cosmetics Shop,Indian Restaurant,History Museum,Hakka Restaurant
9,"Guildwood, Morningside, West Hill",Breakfast Spot,Rental Car Location,Tech Startup,Pizza Place,Mexican Restaurant,Electronics Store,Medical Center,Cosmetics Shop,Department Store,Discount Store


### Cluster neighorboods

In [394]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 1, 1, 1, 1, 2, 1, 1, 1], dtype=int32)

In [389]:
kmeans.labels_

array([1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 1, 0, 3, 1], dtype=int32)

In [396]:
scarborough_data.shape

(17, 5)

In [405]:
# create a new datafrom that include the cluster as well as the top 10 venues of each neighborhood

scarborough_merged = scarborough_data[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how="inner")

scarborough_merged.head() # check the last columns!

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,History Museum,Bar,Vietnamese Restaurant,Convenience Store,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Breakfast Spot,Rental Car Location,Tech Startup,Pizza Place,Mexican Restaurant,Electronics Store,Medical Center,Cosmetics Shop,Department Store,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Convenience Store,Korean Restaurant,Vietnamese Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Athletics & Sports,Thai Restaurant,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Cosmetics Shop,History Museum


In [406]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

#### Cluster 1

In [407]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,0,Pizza Place,Italian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Noodle House,Pharmacy,Chinese Restaurant,Thai Restaurant,Bar,Convenience Store


#### Cluster 2

In [409]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store
2,Scarborough,1,Breakfast Spot,Rental Car Location,Tech Startup,Pizza Place,Mexican Restaurant,Electronics Store,Medical Center,Cosmetics Shop,Department Store,Discount Store
3,Scarborough,1,Coffee Shop,Convenience Store,Korean Restaurant,Vietnamese Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,1,Athletics & Sports,Thai Restaurant,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Cosmetics Shop,History Museum
5,Scarborough,1,Cosmetics Shop,Playground,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,Scarborough,1,Bus Line,Bakery,Intersection,Metro Station,Fast Food Restaurant,Bus Station,Park,Soccer Field,General Entertainment,Cosmetics Shop
8,Scarborough,1,American Restaurant,Motel,Intersection,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,Scarborough,1,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,History Museum,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
11,Scarborough,1,Sandwich Place,Smoke Shop,Middle Eastern Restaurant,Breakfast Spot,Shopping Mall,Paper / Office Supplies Store,Bakery,Auto Garage,General Entertainment,Fried Chicken Joint
12,Scarborough,1,Chinese Restaurant,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint


#### Cluster 3

In [410]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Discount Store,Bus Station,Department Store,Coffee Shop,Train Station,Vietnamese Restaurant,Cosmetics Shop,Indian Restaurant,History Museum,Hakka Restaurant


#### Cluster 4

In [411]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,History Museum,Bar,Vietnamese Restaurant,Convenience Store,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
14,Scarborough,3,Playground,Park,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 5

In [412]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,4,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Bank,Bar,History Museum,Hakka Restaurant,Athletics & Sports
